<a href="https://colab.research.google.com/github/BBARRY-Lee/NLP/blob/main/KoBERT%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인사이드 아웃 캐릭터 기반 감정분석

## 1. 개발 환경 설정

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pwd

'/content'

In [4]:
cd '/content/drive/MyDrive/Inside_out_project'

/content/drive/MyDrive/Inside_out_project


In [5]:
# requirements : https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt
!pip install mxnet
!pip install pandas tqdm
!pip install numpy==1.23.1
!pip install gluonnlp==0.8.0
!pip install sentencepiece
!pip install transformers
!pip install torch

  Using cached gluonnlp-0.8.0-py3-none-any.whl
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━

In [6]:
# https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf 의 kobert_tokenizer 폴더를 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-r7jh8wxj/kobert-tokenizer_483046667d7c4575a48d9405ec80ef0c
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-r7jh8wxj/kobert-tokenizer_483046667d7c4575a48d9405ec80ef0c
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=ad378ea5f99f149560ad13d25ecbc0de16d20427afd4454571379fb8f6b3c163
  Stored in directory: /tmp/pip-ephem-wheel-cache-yr67ufh3/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [7]:
# 라이브러리 세팅
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd

# numpy 버전 확인
print(np.__version__)

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


1.23.1


In [8]:
# GPU 사용
device = torch.device("cuda:0")

# CPU 사용 시
# device = torch.device("cpu")

---

## 2. 데이터셋 로드 및 전처리

In [9]:
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋 불러오기
data = pd.read_csv("./dataset/5차년도_2차.csv", encoding='cp949')

In [27]:
data.head()

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female


In [28]:
data.shape

(19374, 15)

In [29]:
data['상황'].unique()

array(['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust',
       'fear'], dtype=object)

In [30]:
# 감정을 인사이드 아웃 캐릭터에 맞게 매핑
emotion_to_character = {
    'happiness': 'Joy',
    'neutral': 'Joy',     # neutral을 Joy로 매핑
    'sadness': 'Sadness',
    'angry': 'Anger',
    'surprise': 'Joy',    # surprise를 Joy로 매핑
    'disgust': 'Disgust',
    'fear': 'Fear'
}

In [31]:
# 매핑 적용
data['캐릭터'] = data['상황'].map(emotion_to_character)

# 캐릭터를 숫자로 변환
character_to_number = {
    'Joy': 0,
    'Sadness': 1,
    'Anger': 2,
    'Disgust': 3,
    'Fear': 4
}

data['캐릭터_번호'] = data['캐릭터'].map(character_to_number)

In [32]:
data['캐릭터'].unique()

array(['Joy', 'Sadness', 'Anger', 'Disgust', 'Fear'], dtype=object)

In [33]:
data.head()

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별,캐릭터,캐릭터_번호
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female,Joy,0
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female,Joy,0
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female,Joy,0
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female,Joy,0
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female,Joy,0


In [34]:
# [발화문, 상황] data_list 생성
data_list = []
for ques, label in zip(data['발화문'], data['캐릭터_번호']):
    data = []
    data.append(ques)
    data.append(str(label))
    data_list.append(data)

In [35]:
print(data)
print(data_list[:10])

['걱정해줘서 고마워.', '3']
[['헐! 나 이벤트에 당첨 됐어.', '0'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '0'], ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '0'], ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '0'], ['에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.', '0'], ['난 부페 형식의 음식들도 정말 좋아해. 그 것도 좀 알려 줘.', '0'], ['응. 완전히 끝난 거야. 한 달 동안 주말에 쉬지도 못하고 일만 했거든.', '0'], ['신나는 음악 듣는 것도 좋고, 어디 여행 가고 싶고 이 것 저 것 다 해보고 싶어.', '0'], ['친구들도 내 연락 기다리고 있을 텐데 내가 까먹고 있었네?', '0'], ['그래. 일단은 친구들부터 만나서 여행 계획에 대해서 얘기 좀 해봐야 되겠어.', '0']]


---

### 2-1.Split train & test data set

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [38]:
print(len(dataset_train), len(dataset_test))

15499 3875


---

### 2-2.입력 데이터셋 토큰화

In [20]:
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [21]:
# BERT 모델, Vocabulary 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [22]:
# tokenizer 동작확인
inputs = tokenizer("목차를 봤을 때 기억에 오래 남아요", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[   2, 2068, 7389, 6116, 2421, 7088, 1844, 1291, 6896, 3422, 1420, 6999,
            3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [23]:
class BERTSentenceTransform:
    def __init__(self, tokenizer, max_seq_length, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair

    def __call__(self, line):
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer.tokenize(text_b)

        if tokens_b:
            self._truncate_seq_pair(tokens_a, tokens_b, self._max_seq_length - 3)
        else:
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        tokens = []
        tokens.append("[CLS]")
        tokens.extend(tokens_a)
        tokens.append("[SEP]")
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append("[SEP]")
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)
        valid_length = len(input_ids)

        if self._pad:
            padding_length = self._max_seq_length - valid_length
            input_ids.extend([0] * padding_length)  # 0 is the [PAD] token ID
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'), np.array(segment_ids, dtype='int32')

    def _truncate_seq_pair(self, tokens_a, tokens_b, max_length):
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()

In [24]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return len(self.labels)

---

### 2-3.Setting parameters

In [25]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

---

### 2-4.tokenization, int encoding, padding

In [39]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
tok = tokenizer.tokenize

# 데이터셋 생성
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, max_len, True, False)

In [40]:
# torch 형식의 dataset을 만들어주면서 입력 데이터셋의 전처리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


---

## 3. KoBERT 모델 구현

In [41]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes= 5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [42]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [43]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [44]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

---

## 4. Train

In [102]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-102-486c13cb457b>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.10932435095310211 train acc 0.96875
epoch 1 batch id 201 loss 0.2712728977203369 train acc 0.9155006218905473
epoch 1 train acc 0.919945987654321


<ipython-input-102-486c13cb457b>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/61 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9312719555035129


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.021616855636239052 train acc 1.0
epoch 5 batch id 201 loss 0.03261629864573479 train acc 0.9832866915422885
epoch 5 train acc 0.9818672839506173


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.9304596018735363


> epoch 5 train acc 0.9776877572016461

> epoch 5 test acc 0.9208138173302107

---

In [109]:
# 모델 학습이 완료된 후 모델 저장
MODEL_PATH = 'kobert_model.pth'
torch.save(model.state_dict(), MODEL_PATH)
print(f"모델이 {MODEL_PATH}에 저장되었습니다.")

모델이 kobert_model.pth에 저장되었습니다.


## 5. Test

In [107]:
# predict : 학습 모델을 활용하여 다중 분류된 클래스를 출력해주는 함수
def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨(Joy)")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔(Sadness)")
            elif np.argmax(logits) == 2:
                test_eval.append("버럭(Anger)")
            elif np.argmax(logits) == 3:
                test_eval.append("까칠(Disgust)")
            elif np.argmax(logits) == 4:
                test_eval.append("소심(Fear)")

        print(">> 당신의 감정은 " + test_eval[0] + "(이)와 유사하게 느껴집니다.")


In [108]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 너무 우울해.
>> 당신의 감정은 슬픔(Sadness)(이)와 유사하게 느껴집니다.


하고싶은 말을 입력해주세요 : 시험기간이 너무 힘들어.
>> 당신의 감정은 기쁨(Joy)(이)와 유사하게 느껴집니다.


하고싶은 말을 입력해주세요 : 밥이 너무 기대돼. 너무 기뻐 
>> 당신의 감정은 기쁨(Joy)(이)와 유사하게 느껴집니다.


하고싶은 말을 입력해주세요 : 집에 가고 싶다
>> 당신의 감정은 소심(Fear)(이)와 유사하게 느껴집니다.


하고싶은 말을 입력해주세요 : 엘리베이터에 갇혔었어
>> 당신의 감정은 소심(Fear)(이)와 유사하게 느껴집니다.


하고싶은 말을 입력해주세요 : 모기한테 물려서 엄청 고생했어.
>> 당신의 감정은 기쁨(Joy)(이)와 유사하게 느껴집니다.


하고싶은 말을 입력해주세요 : 종강은 언제올까.
>> 당신의 감정은 슬픔(Sadness)(이)와 유사하게 느껴집니다.


하고싶은 말을 입력해주세요 : 0


# 감정분석 기반 영화 추천 시스템

## 1. 데이터셋 로드 및 전처리

### 1-1. 데이터셋 로드

In [10]:
movie_data = pd.read_csv('./dataset/movies.csv')

### 1-2. 감정-장르 매핑 설정

In [11]:
emotion_genre_map = {
    'Joy': ['Comedy', 'Animation', 'Musical', 'Fantasy'],
    'Sadness': ['Drama', 'Romance'],
    'Anger': ['Action', 'Thriller', 'Crime'],
    'Disgust': ['Horror', 'Mystery'],
    'Fear': ['Horror', 'Mystery', 'Thriller']
}

## 2. 감정 분석 결과에 따른 영화 추천 함수 구현

In [13]:
import numpy as np

def recommend_movie(emotion, movie_data, emotion_genre_map):
    if emotion not in emotion_genre_map:
        return "추천할 수 있는 영화가 없습니다.", ""

    # 감정에 해당하는 장르 리스트
    genres = emotion_genre_map[emotion]

    # 해당 장르에 해당하는 영화 필터링
    filtered_movies = movie_data[movie_data['genres'].apply(lambda x: any(genre in x for genre in genres))]

    if filtered_movies.empty:
        return "추천할 수 있는 영화가 없습니다.", ""

    # 무작위로 하나의 영화 선택
    recommended_movie = filtered_movies.sample(n=1).iloc[0]
    return recommended_movie['title'], ""


## 3. Streamlit을 사용하여 웹 애플리케이션 구현

### 3-1. KoBERT 모델 로드

In [46]:
# 모델 로드
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('kobert_model.pth'))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

### 3-2. 감정 분석 및 영화 추천 함수

In [47]:
# 감정 분석 및 영화 추천 함수
def predict_and_recommend(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                emotion = "Joy"
            elif np.argmax(logits) == 1:
                emotion = "Sadness"
            elif np.argmax(logits) == 2:
                emotion = "Anger"
            elif np.argmax(logits) == 3:
                emotion = "Disgust"
            elif np.argmax(logits) == 4:
                emotion = "Fear"

        recommended_movie, poster_url = recommend_movie(emotion, movie_data, emotion_genre_map)
        return emotion, recommended_movie, poster_url

In [48]:
# 감정에 따른 캐릭터 이미지 설정
emotion_character_map = {
    'Joy': 'https://image_url_for_joy',  # 기쁨 이미지 URL
    'Sadness': 'https://image_url_for_sadness',  # 슬픔 이미지 URL
    'Anger': 'https://image_url_for_anger',  # 버럭 이미지 URL
    'Disgust': 'https://image_url_for_disgust',  # 까칠 이미지 URL
    'Fear': 'https://image_url_for_fear'  # 소심 이미지 URL
}

### 3-3. Stramlit UI

In [52]:
# Streamlit 설치
! pip install streamlit

In [55]:
# Streamlit UI
import streamlit as st

st.title("인사이드 아웃 감정 분석 기반 영화 추천 서비스")

user_input = st.text_input("하고싶은 말을 입력해주세요:")

if st.button("분석 및 추천"):
    emotion, movie_title, poster_url = predict_and_recommend(user_input)
    st.write(f"당신의 감정은 {emotion}와 유사하게 느껴집니다.")
    st.write(f"추천 영화: {movie_title}")
    if emotion in emotion_character_map:
        st.image(emotion_character_map[emotion], width=200)
    if poster_url:
        st.image(poster_url)

2024-06-16 12:33:08.606 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-16 12:33:08.611 Session state does not function when running a script without `streamlit run`
